In [6]:
from Grove import Indices
from Grove.Entry import BaseEntry
import numpy as np
import time
import sys

In [ ]:
DIM = 768
NUM_VECTORS = 20
def random_vector() -> np.array:
    return np.float32(np.random.rand(DIM))

def time_function(func, *args, **kwargs) -> float:
    start = time.time()
    func(*args, **kwargs)
    end = time.time()
    return end - start

def get_mem_usage(obj) -> str:
    size_bytes = sys.getsizeof(obj)
    size_mb = size_bytes / (1024 * 1024)
    return f"{size_mb} MB"

def check_equality(a: np.array, b: np.array) -> None:
    np.testing.assert_array_equal(a,b)

q = random_vector()
emb = [random_vector() for i in range(NUM_VECTORS)]

Usage

In [ ]:
from Grove.Extras import HuggingFaceEmbeddings
from api_keys import HUGGINGFACEHUB_API_TOKEN

embeddings = HuggingFaceEmbeddings(HUGGINGFACEHUB_API_TOKEN)

In [ ]:
import fitz
def split_tuples(text_tuples):
    questions = []
    current_question = None
    for flag, color, text in text_tuples:
        if (flag == 16) & (color == 0):  # bolded text flag
            if current_question:
                if questions and current_question['question'] == questions[-1]['question']:
                    # update previous question's answer
                    questions[-1]['answer'] += text
                elif len(text.split()) < 3:  # not a question
                    current_question['answer'] += " " + text
                else:  # new question
                    questions.append(current_question)
                    current_question = {'question': text, 'answer': ''}
            else:
                current_question = {'question': text, 'answer': ''}
        else:  # non-bolded text flag
            if current_question:
                current_question['answer'] += " " + text
    if current_question:
        questions.append(current_question)
    return questions
fname = "/Users/weihern/Documents/Computing Projects/FRIDAY/FRIDAY Working/A500S.pdf"
doc = fitz.open(fname)  # open document
text_tuples = [(s["flags"], s["color"],s["text"]) for page in doc for block in page.get_text("dict")["blocks"] for line in block["lines"] for s in line["spans"]]
questions = split_tuples(text_tuples)
questions = [x["question"] + "\n" + x["answer"] for x in questions if x["answer"] != ""]

In [ ]:
embs = [embeddings(x) for x in questions]

In [ ]:
root = Indices.SVMRootIndex("DashCamSG", 10)
root.create_child("A500S", Indices.HNSWLeafIndex, dim = 768, key = embeddings("A500S"), max_children = 18)
root.insert_all([BaseEntry(embs[i], metadata = {"content": questions[i], "id": i}) for i in range(len(embs))], "A500S")
root.create_child("somenonesnse", Indices.HNSWLeafIndex, dim = 768, key = embeddings("somenonesnse"), max_children = 1000)
root.create_child("somenonesnse2", Indices.HNSWLeafIndex, dim = 768, key = random_vector(), max_children = 1000)
root.create_child("somenonesnse3", Indices.HNSWLeafIndex, dim = 768, key = random_vector(), max_children = 1000)
root.create_child("somenonesnse4", Indices.HNSWLeafIndex, dim = 768, key = random_vector(), max_children = 1000)
root.create_child("somenonesnse5", Indices.HNSWLeafIndex, dim = 768, key = random_vector(), max_children = 1000)
root.create_child("somenonesnse6", Indices.HNSWLeafIndex, dim = 768, key = random_vector(), max_children = 1000)
root.create_child("somenonesnse7", Indices.HNSWLeafIndex, dim = 768, key = random_vector(), max_children = 1000)
root.create_child("somenonesnse8", Indices.HNSWLeafIndex, dim = 768, key = random_vector(), max_children = 1000)
root.create_child("somenonesnse9", Indices.HNSWLeafIndex, dim = 768, key = random_vector(), max_children = 1000)

In [ ]:
query = np.array(embeddings("What is the resoulution of the A500"))
path, res, distances  = root.search(query=query, k = 3)
[x.metadata for x in res[0]]

[{'content': '3. What’s the difference between A500S and A400?\n A500s has a higher resolution for front camera, option for timelapse recording, build in GPS compare to A400 Pro plus is A500s. Global edition is for international use, with English menu & updateable firmware',
  'id': 5},
 {'content': 'What’s the difference between A500S and A800S?\n The main difference between the A800S & A500S would be the recording resolution, the A500S records up to 2.7K resolution, while the A800S records up to 4K. Screen size on A500S is 2 inch, while screen size on A800S is 3 inch.',
  'id': 4},
 {'content': 'What is build in GPS for (A500S / 800S / DDPAI)?\n To see where the footage was taken and route tracking from the footage playback. 5.',
  'id': 8}]

In [ ]:
res[0][0].metadata.get("content")

'3. What’s the difference between A500S and A400?\n A500s has a higher resolution for front camera, option for timelapse recording, build in GPS compare to A400 Pro plus is A500s. Global edition is for international use, with English menu & updateable firmware'

In [ ]:
labels, d = root.children["A500S"].index.knn_query(query, k = 3)

In [ ]:
get_mem_usage(root)

'4.57763671875e-05 MB'

In [ ]:
print(root.get_schema())

SVMRootIndex(name = DashCamSG, num_children = 10)
    HNSWLeafIndex(name = somenonesnse9, dim = 768, current_count = 0, searchable = True)
    HNSWLeafIndex(name = somenonesnse8, dim = 768, current_count = 0, searchable = True)
    HNSWLeafIndex(name = somenonesnse7, dim = 768, current_count = 0, searchable = True)
    HNSWLeafIndex(name = somenonesnse6, dim = 768, current_count = 0, searchable = True)
    HNSWLeafIndex(name = somenonesnse5, dim = 768, current_count = 0, searchable = True)
    HNSWLeafIndex(name = somenonesnse4, dim = 768, current_count = 0, searchable = True)
    HNSWLeafIndex(name = somenonesnse3, dim = 768, current_count = 0, searchable = True)
    HNSWLeafIndex(name = somenonesnse2, dim = 768, current_count = 0, searchable = True)
    HNSWLeafIndex(name = somenonesnse, dim = 768, current_count = 0, searchable = True)
    HNSWLeafIndex(name = A500S, dim = 768, current_count = 18, searchable = True)



In [ ]:
root.save_to_disk()

In [ ]:
del root

In [ ]:
root = Indices.SVMRootIndex.load_from_disk("DashCamSG")
path, res, distances  = root.search(query=query, k = 3)
[x.metadata for x in res[0]]

[{'content': '3. What’s the difference between A500S and A400?\n A500s has a higher resolution for front camera, option for timelapse recording, build in GPS compare to A400 Pro plus is A500s. Global edition is for international use, with English menu & updateable firmware',
  'id': 5},
 {'content': 'What’s the difference between A500S and A800S?\n The main difference between the A800S & A500S would be the recording resolution, the A500S records up to 2.7K resolution, while the A800S records up to 4K. Screen size on A500S is 2 inch, while screen size on A800S is 3 inch.',
  'id': 4},
 {'content': 'What is build in GPS for (A500S / 800S / DDPAI)?\n To see where the footage was taken and route tracking from the footage playback. 5.',
  'id': 8}]